In [1]:
import sys
import os
import itertools
sys.path.append(os.path.abspath('../'))
from state_constrainer_YY import state_constrainer
from IPython.utils import io
import time

In [2]:
m = state_constrainer('boundedStackInvariant.sol')
m.take_snapshot()

# -- Deploying Contract boundedStackInvariant.sol
(level _is_sat_z3_call) took 0.0004260540008544922 seconds (sat)
# -- Contract Deployed      (took 23.322353839874268 seconds)


In [3]:
metodos = ["push","pop"]
precondiciones = [x+"_precondition" for x in metodos]
def call_pre():
    for cond in precondiciones:
        m.callContractFunction(cond,tx_sender=m.witness_account)

In [4]:
states = list(itertools.product([0,1],repeat=len(metodos)))
def repr_state(state):
    text = ""
    for x,method in zip(state,metodos):
        text += '_'+method if x else ""
    if text == "":
        text = "vacio"
    return text

In [5]:
call_pre()

In [6]:
found = []
with io.capture_output() as captured:    
    for state in states:
        count = m.generateTestCases(keys=precondiciones,
                                    targets=state,
                                    testcaseName=f"STATE_{repr_state(state)}")
        if count > 0:
            found.append(f"found {count} testcases that reach {repr_state(state)} initial state")
        else:
            found.append(f"found no testcases for {repr_state(state)} initial state")

In [7]:
for line in found:
    print(line)

found True testcases that reach vacio initial state
found no testcases for _pop initial state
found True testcases that reach _push initial state
found no testcases for _push_pop initial state


In [8]:
setter_id = m.nameToFuncId['setter']
types = m.contract_metadata.get_func_argument_types(setter_id)
print(types)
types = types[:-2]+'10'+types[-2:]
print(types)

(uint256,uint256,uint256[])
(uint256,uint256,uint256[10])


In [9]:
setterArgs = m.manticore.make_symbolic_arguments(types)
print(setterArgs)

(<BitVecVariable(SVALUE_1) at 7f18cbd37440>, <BitVecVariable(SVALUE_2) at 7f18cbd372c0>, [<BitVecVariable(SVALUE_3) at 7f18cbd35f80>, <BitVecVariable(SVALUE_4) at 7f18cbd375c0>, <BitVecVariable(SVALUE_5) at 7f18cbd377c0>, <BitVecVariable(SVALUE_6) at 7f18cbd36880>, <BitVecVariable(SVALUE_7) at 7f18cbd37900>, <BitVecVariable(SVALUE_8) at 7f18cbd37b00>, <BitVecVariable(SVALUE_9) at 7f18cbd37e00>, <BitVecVariable(SVALUE_10) at 7f18cbd37f40>, <BitVecVariable(SVALUE_11) at 7f18cbd37c40>, <BitVecVariable(SVALUE_12) at 7f18cbd37a00>])


In [10]:
m.goto_snapshot()
m.take_snapshot()
start_setter = time.time()
with io.capture_output() as captured:
    m.callContractFunction("setter",call_args = setterArgs, tx_value = 0, tx_sender = m.witness_account)
print(time.time()-start_setter)

150.1890676021576


In [11]:
start_inv = time.time()
with io.capture_output() as captured:
    m.constrainTo("invariant",True)
print(time.time()-start_inv)

171.2444624900818


In [12]:
call_pre()
m.take_snapshot()

(level _is_sat_z3_call) took 0.2866075038909912 seconds (sat)
(level _is_sat_z3_call) took 0.16275572776794434 seconds (unsat)
(level _is_sat_z3_call) took 0.23439240455627441 seconds (sat)
(level _is_sat_z3_call) took 0.0052471160888671875 seconds (unsat)
(level _is_sat_z3_call) took 0.04027295112609863 seconds (sat)
(level _is_sat_z3_call) took 0.05490612983703613 seconds (sat)


In [13]:
with io.capture_output() as captured:    
    m.callContractFunction("push")
call_pre()

(level _is_sat_z3_call) took 0.14525461196899414 seconds (sat)
(level _is_sat_z3_call) took 0.09169292449951172 seconds (sat)
(level _is_sat_z3_call) took 0.05942940711975098 seconds (sat)
(level _is_sat_z3_call) took 0.09732604026794434 seconds (sat)
(level _is_sat_z3_call) took 0.09786868095397949 seconds (sat)(level _is_sat_z3_call) took 0.07766580581665039 seconds (sat)

(level _is_sat_z3_call) took 0.08969402313232422 seconds (sat)
(level _is_sat_z3_call) took 0.18428254127502441 seconds (sat)


In [14]:
def repr_transition(state1,state2,metodo):
    return f"{repr_state(state1)}->{metodo}->{repr_state(state2)}"

In [15]:
ini_state = (1,0) #_push
found = []
with io.capture_output() as captured:    
    for fin_state in states:
        count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                    targets=(ini_state+fin_state),
                                    testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'push')}")
        if count > 0:
            found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'push')}")
        else:
            found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'push')}")

In [16]:
for line in found:
    print(line)

found no testcases for transition _push->push->vacio
found no testcases for transition _push->push->_pop
found no testcases for transition _push->push->_push
found no testcases for transition _push->push->_push_pop


In [17]:
m.goto_snapshot()
m.take_snapshot()
with io.capture_output() as captured:
    m.callContractFunction("push")
call_pre()

(level _is_sat_z3_call) took 0.05116415023803711 seconds (sat)
(level _is_sat_z3_call) took 0.08744955062866211 seconds (sat)
(level _is_sat_z3_call) took 0.07607555389404297 seconds (sat)
(level _is_sat_z3_call) took 0.08217263221740723 seconds (sat)
(level _is_sat_z3_call) took 0.05621647834777832 seconds (sat)
(level _is_sat_z3_call) took 0.12699675559997559 seconds (sat)
(level _is_sat_z3_call) took 0.16909313201904297 seconds (sat)
(level _is_sat_z3_call) took 0.11522364616394043 seconds (sat)


In [18]:
allowing_states = [(1,0),(1,1)] #_push, #_push_pop
found = []
with io.capture_output() as captured:  
    for ini_state in allowing_states:
        for fin_state in states:
            count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                        targets=(ini_state+fin_state),
                                        testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'push')}")
            if count > 0:
                found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'push')}")
            else:
                found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'push')}")

In [19]:
for line in found:
    print(line)

found no testcases for transition _push->push->vacio
found no testcases for transition _push->push->_pop
found no testcases for transition _push->push->_push
found no testcases for transition _push->push->_push_pop
found no testcases for transition _push_pop->push->vacio
found True testcases for transition _push_pop->push->_pop
found no testcases for transition _push_pop->push->_push
found True testcases for transition _push_pop->push->_push_pop


In [20]:
m.goto_snapshot()
m.take_snapshot()
start_pop = time.time()
with io.capture_output() as captured:
    m.callContractFunction("pop")
    call_pre()
print(time.time()-start_pop)

353.8847575187683


In [21]:
ini_states = [(1,1),(0,1)] #_push_pop , #_pop
found = []

for ini_state in ini_states:
    for fin_state in states:
        with io.capture_output() as captured:    
            count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                        targets=(ini_state+fin_state),
                                        testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'pop')}")
            if count > 0:
                found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'pop')}")
            else:
                found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'pop')}")

In [22]:
for line in found:
    print(line)

found no testcases for transition _push_pop->pop->vacio
found no testcases for transition _push_pop->pop->_pop
found no testcases for transition _push_pop->pop->_push
found True testcases for transition _push_pop->pop->_push_pop
found no testcases for transition _pop->pop->vacio
found True testcases for transition _pop->pop->_pop
found no testcases for transition _pop->pop->_push
found no testcases for transition _pop->pop->_push_pop
